In [ ]:
!pip install -q cassio datasets langchain openai tiktoken
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
from datasets import load_dataset
import cassio

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

In [ ]:
ASTRA_DB_APPLICATION_TOKEN="please add your db+application_token"
ASTRA_DB_ID="please add you db id from datastak"
OPENAI_API_KEY="please add your openai api key"

In [ ]:
pdfreader=PdfReader("bugdet_speech.pdf")
raw_text = " "
for i,page in enumerate(pdfreader.pages):
  content=page.extract_text()
  if content:
    raw_text+=content

In [ ]:
raw_text

" India's Union Budget is an annual financial statement presented by the Finance Minister. It outlines\nthe government's revenue and expenditure for the upcoming fiscal year. The budget includes\nallocations to various sectors like education, healthcare, infrastructure, and defense. It also\nproposes tax reforms, subsidy plans, and economic policies to drive national growth. This financial\nroadmap helps in planning the nations developmental priorities and reflects the government's vision.\nOne of the major highlights of the budget is its focus on sustainable and inclusive growth. The\ngovernment often proposes new schemes targeting agriculture, rural development, and employment\ngeneration. Infrastructure development is emphasized through investments in roads, railways, and\nurban development projects. The budget also outlines steps for fiscal consolidation, aiming to\nreduce the fiscal deficit and ensure macroeconomic stability.\nIn recent years, the digital economy has been another 

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [ ]:
llm= OpenAI(openai_api_key=OPENAI_API_KEY,temprature=0)
embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

<ipython-input-18-43aeca7be4f0>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm= OpenAI(openai_api_key=OPENAI_API_KEY)
<ipython-input-18-43aeca7be4f0>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [ ]:
astra_vector_store=Cassandra(
    embedding=embeddings,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,

)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size=400,
    chunk_overlap=100,
    length_function=len,

)
texts=text_splitter.split_text(raw_text)
texts[:50]

["India's Union Budget is an annual financial statement presented by the Finance Minister. It outlines\nthe government's revenue and expenditure for the upcoming fiscal year. The budget includes\nallocations to various sectors like education, healthcare, infrastructure, and defense. It also\nproposes tax reforms, subsidy plans, and economic policies to drive national growth. This financial",
 "proposes tax reforms, subsidy plans, and economic policies to drive national growth. This financial\nroadmap helps in planning the nations developmental priorities and reflects the government's vision.\nOne of the major highlights of the budget is its focus on sustainable and inclusive growth. The\ngovernment often proposes new schemes targeting agriculture, rural development, and employment",
 'government often proposes new schemes targeting agriculture, rural development, and employment\ngeneration. Infrastructure development is emphasized through investments in roads, railways, and\nurban deve

In [ ]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines" % len(texts[:50]))
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 14 headlines


In [ ]:
first_question = True

while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == 'quit':
        break

    if query_text == "":
        continue

    first_question = False

    print(f"\nQUESTION: \n\n{query_text}")
    answer = astra_vector_index.query(query_text, llm=llm).strip()

    print(f"\nANSWER:\n\n{answer}")

    print("\nFIRST DOCUMENTS By RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print(f"[{score:.04f}] %s...\n" % (doc.page_content[:84]))


Enter your question (or type 'quit' to exit): what is the current gdp?

QUESTION: 

what is the current gdp?



ANSWER:

I don't know.

FIRST DOCUMENTS By RELEVANCE:


[0.8862] for the upcoming fiscal year. The budget includes allocations to various sectors lik...

[0.8849] In recent years, the digital economy has been another major focus. Budget announceme...

[0.8848] year ahead. Public and industry reactions to the budget are closely watched, and the...

[0.8843] proposes tax reforms, subsidy plans, and economic policies to drive national growth....


What's your next question (or type 'quit' to exit): what is the major budget

QUESTION: 

what is the major budget



ANSWER:

The major budget is an annual financial statement presented by the Finance Minister that outlines the government's revenue and expenditure for the upcoming fiscal year. It includes allocations to various sectors and proposes tax reforms, subsidy plans, and economic policies to drive national growth. Additionally, it helps in planning the nation's developmental priorities and reflects the government's vision.

FIRST DOCUMENTS By RELEVANCE:


[0.9108] priorities and reflects the government's vision.
One of the major highlights of the ...

[0.9089] for the upcoming fiscal year. The budget includes allocations to various sectors lik...

[0.9084] India's Union Budget is an annual financial statement presented by the Finance Minis...

[0.9029] proposes tax reforms, subsidy plans, and economic policies to drive national growth....


What's your next question (or type 'quit' to exit): what is the major budget focus on

QUESTION: 

what is the major budget focus on



ANSWER:

The major budget focus is on sustainable and inclusive growth, particularly through investments in agriculture, rural development, employment generation, infrastructure development, and the digital economy.

FIRST DOCUMENTS By RELEVANCE:


[0.9331] priorities and reflects the government's vision.
One of the major highlights of the ...

[0.9299] generation. Infrastructure development is emphasized through investments in roads, r...

[0.9232] for the upcoming fiscal year. The budget includes allocations to various sectors lik...

[0.9187] credit schemes and tax incentives to promote entrepreneurship.
Health and education ...

